In [115]:
import torch.optim as optim
from torchvision import models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.tensorboard import SummaryWriter


In [116]:
# Hyperparameters
learning_rate = 0.0001
batch_size = 50
num_epochs = 4 #alter this afterwards
momentum = 0.9
loss_function = nn.CrossEntropyLoss()

# Architecture
num_classes = 10


In [117]:

train_transforms = transforms.Compose([transforms.Resize((70, 70)),
                                       transforms.RandomCrop((64, 64)),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.Resize((70, 70)),
                                      transforms.CenterCrop((64, 64)),
                                      transforms.ToTensor()])



train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


Training-Testing Functions

In [118]:
writer = SummaryWriter()

In [123]:
def train_model(data_loader,network, optimizer, criterion):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    network.to(device)
    network.train()
    for epoch in range(num_epochs):
        running_loss = epoch_loss = 0.
        for i, data in enumerate(data_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            epoch_loss += loss.item()

            if i % 100 == 99:
                print('[Epoch %d, Batch %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0
        writer.add_scalar(f'Loss/train:', epoch_loss / len(data_loader), epoch)
        print(f"[{epoch + 1}] loss: {epoch_loss / len(data_loader):.3f}")
    writer.flush()
    return network

def test_model(data_loader, network):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    network.to(device)
    network.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = alexnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy: %.2f %%' % (100 * correct / total))

Fine tuning the model

In [120]:
# alexnet = models.alexnet(pretrained=True)
alexnet = models.alexnet(weights='AlexNet_Weights.DEFAULT')


# Add a new fully connected layer to the classifier
alexnet.classifier.add_module('6', nn.Linear(4096, 10))


optimizer = optim.SGD(alexnet.parameters(), lr=learning_rate, momentum=momentum)

In [121]:

# Train the model for a few epochs
trained_net = train_model(train_loader, alexnet,optimizer, loss_function)


[Epoch 1, Batch   100] loss: 1.845
[Epoch 1, Batch   200] loss: 1.446
[Epoch 1, Batch   300] loss: 1.331
[Epoch 1, Batch   400] loss: 1.284
[Epoch 1, Batch   500] loss: 1.225
[Epoch 1, Batch   600] loss: 1.176
[Epoch 1, Batch   700] loss: 1.138
[Epoch 1, Batch   800] loss: 1.124
[Epoch 1, Batch   900] loss: 1.078
[Epoch 1, Batch  1000] loss: 1.101
[1] loss: 0.000
[Epoch 2, Batch   100] loss: 1.038
[Epoch 2, Batch   200] loss: 1.012
[Epoch 2, Batch   300] loss: 0.998
[Epoch 2, Batch   400] loss: 1.016
[Epoch 2, Batch   500] loss: 0.981
[Epoch 2, Batch   600] loss: 0.960
[Epoch 2, Batch   700] loss: 0.976
[Epoch 2, Batch   800] loss: 0.955
[Epoch 2, Batch   900] loss: 0.952
[Epoch 2, Batch  1000] loss: 0.940
[2] loss: 0.000
[Epoch 3, Batch   100] loss: 0.919
[Epoch 3, Batch   200] loss: 0.894
[Epoch 3, Batch   300] loss: 0.909
[Epoch 3, Batch   400] loss: 0.873
[Epoch 3, Batch   500] loss: 0.898
[Epoch 3, Batch   600] loss: 0.894
[Epoch 3, Batch   700] loss: 0.850
[Epoch 3, Batch   800] 

In [122]:
# Evaluate the model on the test set

test = test_model(test_loader, trained_net)


Test accuracy: 72.52 %


Feature Extraction

In [126]:
# Load the pre-trained AlexNet model
# alexnet = models.alexnet(pretrained=True)
alexnet = models.alexnet(weights='AlexNet_Weights.DEFAULT')


# Freeze all the layers in the feature extractor
for param in alexnet.features.parameters():
    param.requires_grad = False

# Add a new fully connected layer to the classifier
alexnet.classifier.add_module('6', nn.Linear(4096, 10))

optimizer = optim.SGD(alexnet.parameters(), lr=learning_rate, momentum=momentum)
# Train the model for a few epochs
trained_net = train_model(train_loader,alexnet, optimizer, loss_function)

# Evaluate the model on the test set

test = test_model(test_loader, trained_net)


[Epoch 1, Batch   100] loss: 1.860
[Epoch 1, Batch   200] loss: 1.514
[Epoch 1, Batch   300] loss: 1.467
[Epoch 1, Batch   400] loss: 1.399
[Epoch 1, Batch   500] loss: 1.396
[Epoch 1, Batch   600] loss: 1.381
[Epoch 1, Batch   700] loss: 1.344
[Epoch 1, Batch   800] loss: 1.343
[Epoch 1, Batch   900] loss: 1.316
[Epoch 1, Batch  1000] loss: 1.322
[1] loss: 1.434
[Epoch 2, Batch   100] loss: 1.312
[Epoch 2, Batch   200] loss: 1.280
[Epoch 2, Batch   300] loss: 1.275
[Epoch 2, Batch   400] loss: 1.310
[Epoch 2, Batch   500] loss: 1.289
[Epoch 2, Batch   600] loss: 1.283
[Epoch 2, Batch   700] loss: 1.266
[Epoch 2, Batch   800] loss: 1.274
[Epoch 2, Batch   900] loss: 1.275


KeyboardInterrupt: 